# Read DWD CDC Time Series, Merge with Station Description and Append 

The main idea behind this activity is to reformat and merge time series (here we use hourly precipitation) from the DWD Climate Data Center in such a way that it can be used with the **QGIS time manager extension**. 

This extension allows to filter an attribute table of a vector layer (e.g. points representing precipitation stations plus precipitation data) with a time stamp column. The extension limits the attribute table to the records matching the particular time stamp provided by the time manager extension (e.g. by the user moving the time slider). This selected subset of the attribute table is then used to change the sympology of the vector layer according to the variable of interest (e.g. precipitation rate).

The QGIS time manager extension approach is a bit brute force, because each individual measurement at a station at a given time is one feature (row in the table), i.e. a time series at station X with hourly resolution for a day (24 values) entails 24 different features with the same station id and the corresponding coordinates but different times. As of now this 1:n relationship can only be realized by importing a CSV file with the according structure. 

(At least I was not able to generate the required view on a 1:n relationship by merging a point vector layer with precipitation station locations and an imported CSV time series table.)

The final data format is a concatenation of time series together with geographic location in 2D (e.g. lat, lon). The required data format looks principly like this:


| station_id |        name        |   lat   |   lon  |        meas_time       | prec_rate |
|:----------:|:------------------:|:-------:|:------:|:----------------------:|:---------:|
|        ... | ...                |     ... |    ... |                    ... |       ... |
|       1595 | Gelsenkirchen-Buer | 51.5762 | 7.0652 | 2018-12-07T08:00:00UTC |       1.5 |
|       1595 | Gelsenkirchen-Buer | 51.5762 | 7.0652 | 2018-12-07T09:00:00UTC |       1.7 |
|       1595 | Gelsenkirchen-Buer | 51.5762 | 7.0652 | 2018-12-07T10:00:00UTC |       0.1 |
|        ... | ...                |     ... |    ... |                    ... |       ... |
|      13670 | Duisburg-Baerl     | 51.5088 | 6.7018 | 2018-12-07T08:00:00UTC |       0.8 |
|      13670 | Duisburg-Baerl     | 51.5088 | 6.7018 | 2018-12-07T09:00:00UTC |       0.4 |
|      13670 | Duisburg-Baerl     | 51.5088 | 6.7018 | 2018-12-07T10:00:00UTC |       0.0 |
|        ... | ...                |     ... |    ... |                    ... |       ... |


(Table generated with https://www.tablesgenerator.com/markdown_tables)

To achieve this the precipitation time series (station_id, meas_time, prec_rate) have to be merged with the station metadata (station_id, lat, lon) coming from the a CSV file generated in an earlier activity. We use Pandas to read, join and append the data to generate the final CSV file to be imported as a point layer to QGIS. 

This final data format is far from being optimal because of large size and highly redundant information. This is a challenge for QGIS which loses responsiveness with large data. To jsut show the principle it is advisable to limit to size of the problem. 

The following filters (selection criteria) are applied:

  * Precipitation stations in NRW only (approx. 127 stations) 
  * Hourly precipitation data
  * Time interval from 2018-12-01 to last date in precipitation data set 
  
Still: 40 days * 24 hrs / day * 127 stations = 121920 records leading to 121920 features in a point layer in QGIS. 

In fact, the resulting number of records is arround 91000. The reason might be that not all stations in the station list have time series. This has to be checked carefully.

## FTP Connection

### Connection Parameters

In [1]:
server = "opendata.dwd.de"
user   = "anonymous"
passwd = ""

### FTP Directory Definition and Station Description Filename Pattern

In [2]:
# The topic of interest.
topic_dir = "/daily/more_precip/historical/"
#topic_dir = "/annual/kl/historical/"

# This is the search pattern common to ALL station description file names 
station_desc_pattern = "_Beschreibung_Stationen.txt"

# Below this directory tree node all climate data are stored.
ftp_climate_data_dir = "/climate_environment/CDC/observations_germany/climate/"
ftp_dir =  ftp_climate_data_dir + topic_dir

### Local Directories

In [3]:
local_ftp_dir         = "data/original/DWD/"      # Local directory to store local ftp data copies, the local data source or input data. 
local_ftp_station_dir = local_ftp_dir + topic_dir # Local directory where local station info is located
local_ftp_ts_dir      = local_ftp_dir + topic_dir # Local directory where time series downloaded from ftp are located

local_generated_dir   = "data/generated/DWD/" # The generated of derived data in contrast to local_ftp_dir
local_station_dir     = local_generated_dir + topic_dir # Derived station data, i.e. the CSV file
local_ts_merged_dir   = local_generated_dir + topic_dir # Parallel merged time series, wide data frame with one TS per column
local_ts_appended_dir = local_generated_dir + topic_dir # Serially appended time series, long data frame for QGIS TimeManager Plugin

In [4]:
print(local_ftp_dir)
print(local_ftp_station_dir)
print(local_ftp_ts_dir)
print()
print(local_generated_dir)
print(local_station_dir)
print(local_ts_merged_dir)
print(local_ts_appended_dir)

data/original/DWD/
data/original/DWD//daily/more_precip/historical/
data/original/DWD//daily/more_precip/historical/

data/generated/DWD/
data/generated/DWD//daily/more_precip/historical/
data/generated/DWD//daily/more_precip/historical/
data/generated/DWD//daily/more_precip/historical/


In [5]:
import os
os.makedirs(local_ftp_dir,exist_ok = True) # it does not complain if the dir already exists.
os.makedirs(local_ftp_station_dir,exist_ok = True)
os.makedirs(local_ftp_ts_dir,exist_ok = True)

os.makedirs(local_generated_dir,exist_ok = True)
os.makedirs(local_station_dir,exist_ok = True)
os.makedirs(local_ts_merged_dir,exist_ok = True)
os.makedirs(local_ts_appended_dir,exist_ok = True)

### FTP Connect

In [6]:
import ftplib
ftp = ftplib.FTP(server)
res = ftp.login(user=user, passwd = passwd)
print(res)

230 Login successful.


In [7]:
ret = ftp.cwd(".")

In [8]:
#ftp.quit()

### FTP Grab File Function

In [9]:
def grabFile(ftpfullname,localfullname):
    try:
        ret = ftp.cwd(".") # A dummy action to chack the connection and to provoke an exception if necessary.
        localfile = open(localfullname, 'wb')
        ftp.retrbinary('RETR ' + ftpfullname, localfile.write, 1024)
        localfile.close()
    
    except ftplib.error_perm:
        print("FTP ERROR. Operation not permitted. File not found?")

    except ftplib.error_temp:
        print("FTP ERROR. Timeout.")

    except ConnectionAbortedError:
        print("FTP ERROR. Connection aborted.")



### Generate Pandas Dataframe from FTP Directory Listing

In [10]:
import pandas as pd
import os

def gen_df_from_ftp_dir_listing(ftp, ftpdir):
    lines = []
    flist = []
    try:    
        res = ftp.retrlines("LIST "+ftpdir, lines.append)
    except:
        print("Error: ftp.retrlines() failed. ftp timeout? Reconnect!")
        return
        
    if len(lines) == 0:
        print("Error: ftp dir is empty")
        return
    
    for line in lines:
#        print(line)
        [ftype, fsize, fname] = [line[0:1], int(line[31:42]), line[56:]]
#        itemlist = [line[0:1], int(line[31:42]), line[56:]]
#        flist.append(itemlist)
        
        fext = os.path.splitext(fname)[-1]
        
        if fext == ".zip":
            station_id = int(fname.split("_")[2])
        else:
            station_id = -1 
        
        flist.append([station_id, fname, fext, fsize, ftype])
        
        

    df_ftpdir = pd.DataFrame(flist,columns=["station_id", "name", "ext", "size", "type"])
    return(df_ftpdir)

In [11]:
df_ftpdir = gen_df_from_ftp_dir_listing(ftp, ftp_dir)

In [12]:
df_ftpdir.head()

,station_id,name,ext,size,type
0,-1,BESCHREIBUNG_obsgermany_climate_daily_more_pre...,.pdf,72261,-
1,-1,DESCRIPTION_obsgermany_climate_daily_more_prec...,.pdf,71026,-
2,-1,RR_Tageswerte_Beschreibung_Stationen.txt,.txt,1202717,-
3,1,tageswerte_RR_00001_19120101_19860630_hist.zip,.zip,113547,-
4,2,tageswerte_RR_00002_19510101_20061231_hist.zip,.zip,85314,-


### Dataframe with TS Zip Files

In [13]:
#df_ftpdir["ext"]==".zip"
df_zips = df_ftpdir[df_ftpdir["ext"]==".zip"]
df_zips.set_index("station_id", inplace = True)
df_zips.head()

,name,ext,size,type
station_id,,,,
1,tageswerte_RR_00001_19120101_19860630_hist.zip,.zip,113547,-
2,tageswerte_RR_00002_19510101_20061231_hist.zip,.zip,85314,-
3,tageswerte_RR_00003_18910101_20110331_hist.zip,.zip,172281,-
4,tageswerte_RR_00004_19510101_19791031_hist.zip,.zip,47378,-
6,tageswerte_RR_00006_19821101_20191231_hist.zip,.zip,51877,-


### Download the Station Description File

In [14]:
station_fname = df_ftpdir[df_ftpdir['name'].str.contains(station_desc_pattern)]["name"].values[0]
print(station_fname)

# ALternative
#station_fname2 = df_ftpdir[df_ftpdir["name"].str.match("^.*Beschreibung_Stationen.*txt$")]["name"].values[0]
#print(station_fname2)

RR_Tageswerte_Beschreibung_Stationen.txt


In [15]:
print("grabFile: ")
print("From: " + ftp_dir + station_fname)
print("To:   " + local_ftp_station_dir + station_fname)
grabFile(ftp_dir + station_fname, local_ftp_station_dir + station_fname)

grabFile: 
From: /climate_environment/CDC/observations_germany/climate//daily/more_precip/historical/RR_Tageswerte_Beschreibung_Stationen.txt
To:   data/original/DWD//daily/more_precip/historical/RR_Tageswerte_Beschreibung_Stationen.txt


In [16]:
# extract column names. They are in German (de)
# We have to use codecs because of difficulties with character encoding (German Umlaute)
import codecs

def station_desc_txt_to_csv(txtfile, csvfile):
    file = codecs.open(txtfile,"r","utf-8")
    r = file.readline()
    file.close()
    colnames_de = r.split()
    colnames_de
    
    translate = \
    {'Stations_id':'station_id',
     'von_datum':'date_from',
     'bis_datum':'date_to',
     'Stationshoehe':'altitude',
     'geoBreite': 'latitude',
     'geoLaenge': 'longitude',
     'Stationsname':'name',
     'Bundesland':'state'}
    
    colnames_en = [translate[h] for h in colnames_de]
    
    # Skip the first two rows and set the column names.
    df = pd.read_fwf(txtfile,skiprows=2,names=colnames_en, parse_dates=["date_from","date_to"],index_col = 0)
    
    # write csv
    df.to_csv(csvfile, sep = ";")
    return(df)

In [17]:
basename = os.path.splitext(station_fname)[0]
df_stations = station_desc_txt_to_csv(local_ftp_station_dir + station_fname, local_station_dir + basename + ".csv")
df_stations.head()

,date_from,date_to,altitude,latitude,longitude,name,state
station_id,,,,,,,
1,1912-01-01,1986-06-30,478,47.8413,8.8493,Aach,Baden-Württemberg
2,1951-01-01,2006-12-31,138,50.8066,6.0996,Aachen (Kläranlage),Nordrhein-Westfalen
3,1891-01-01,2011-03-31,202,50.7827,6.0941,Aachen,Nordrhein-Westfalen
4,1951-01-01,1979-10-31,243,50.7683,6.1207,Aachen-Brand,Nordrhein-Westfalen
6,1982-11-01,2020-06-30,455,48.8361,10.0598,Aalen-Unterrombach,Baden-Württemberg


### Select Stations Located in NRW from Station Description Dataframe

In [18]:
isOperational = df_stations.date_to.max() 

station_ids_selected = df_stations[df_stations['state'].str.contains("Bayern")].index 
station_ids_selected
#isOperational


Int64Index([   10,    15,    16,    17,    24,    27,    29,    37,    46,
               48,
            ...
            15930, 15943, 15945, 15953, 15965, 15977, 19087, 19120, 19127,
            19128],
           dtype='int64', name='station_id', length=1298)

In [19]:
# Create variable with TRUE if state is Nordrhein-Westfalen
isNRW = df_stations['state'] == "Bayern"

# Create variable with TRUE if date_to is latest date (indicates operation up to now)
isOperational = df_stations['date_to'] == df_stations.date_to.max() 

isBefore1950 = df_stations['date_from'] < '1980'


# select on both conditions
dfNRW = df_stations[isNRW & isOperational & isBefore1950]

#print("Number of stations in NRW: \n", dfNRW.count())
dfNRW

,date_from,date_to,altitude,latitude,longitude,name,state
station_id,,,,,,,
29,1951-01-01,2020-06-30,260,49.7175,10.9101,Adelsdorf (Kläranlage),Bayern
46,1941-01-01,2020-06-30,325,48.9450,12.4639,Aholfing,Bayern
55,1951-01-01,2020-06-30,509,47.8780,12.0239,"Aibling, Bad-Ellmosen",Bayern
57,1978-01-01,2020-06-30,461,48.4799,11.0521,Hollenbach/Bayern-Schönbach,Bayern
63,1951-01-01,2020-06-30,747,47.8172,10.5374,Aitrang,Bayern
...,...,...,...,...,...,...,...
5941,1899-01-01,2020-06-30,686,47.6754,12.4698,Reit im Winkl,Bayern
6284,1941-01-01,2020-06-30,420,49.2289,10.5044,Herrieden (Kläranlage),Bayern
6295,1931-01-01,2020-06-30,650,47.8030,11.8250,Miesbach (Kläranlage),Bayern


In [20]:
print(df_zips)

                                                      name   ext    size type
station_id                                                                   
1           tageswerte_RR_00001_19120101_19860630_hist.zip  .zip  113547    -
2           tageswerte_RR_00002_19510101_20061231_hist.zip  .zip   85314    -
3           tageswerte_RR_00003_18910101_20110331_hist.zip  .zip  172281    -
4           tageswerte_RR_00004_19510101_19791031_hist.zip  .zip   47378    -
6           tageswerte_RR_00006_19821101_20191231_hist.zip  .zip   51877    -
...                                                    ...   ...     ...  ...
19096       tageswerte_RR_19096_19530401_20020630_hist.zip  .zip   73802    -
19120       tageswerte_RR_19120_20190701_20190923_hist.zip  .zip    9475    -
19123       tageswerte_RR_19123_20190701_20191231_hist.zip  .zip  294489    -
19125       tageswerte_RR_19125_20190901_20191231_hist.zip  .zip    7462    -
19127       tageswerte_RR_19127_20191001_20191231_hist.zip  .zip

### Download TS Data from FTP Server

Problem: Not all stations listed in the station description file are associated with a time series (zip file)! The stations in the description file and the set of stations whoch are TS data provided for (zip files) do not match perfectly.  

In [21]:
list(dfNRW.index)

[29,
 46,
 55,
 57,
 63,
 103,
 118,
 128,
 142,
 145,
 146,
 151,
 160,
 163,
 202,
 203,
 205,
 217,
 223,
 227,
 232,
 262,
 282,
 318,
 319,
 320,
 323,
 349,
 359,
 360,
 376,
 447,
 475,
 509,
 515,
 516,
 522,
 552,
 562,
 572,
 606,
 647,
 653,
 734,
 745,
 759,
 789,
 801,
 803,
 807,
 808,
 811,
 812,
 851,
 857,
 867,
 885,
 922,
 933,
 952,
 973,
 983,
 985,
 1023,
 1094,
 1103,
 1107,
 1119,
 1132,
 1140,
 1143,
 1161,
 1235,
 1237,
 1248,
 1256,
 1264,
 1279,
 1309,
 1313,
 1322,
 1325,
 1332,
 1343,
 1357,
 1365,
 1366,
 1374,
 1382,
 1401,
 1432,
 1458,
 1459,
 1473,
 1511,
 1528,
 1534,
 1542,
 1550,
 1575,
 1577,
 1578,
 1587,
 1588,
 1601,
 1642,
 1652,
 1682,
 1689,
 1729,
 1730,
 1734,
 1751,
 1764,
 1844,
 1846,
 1851,
 1886,
 1900,
 1909,
 1933,
 1938,
 1950,
 1997,
 2051,
 2064,
 2079,
 2103,
 2109,
 2113,
 2123,
 2132,
 2168,
 2178,
 2222,
 2255,
 2261,
 2268,
 2274,
 2290,
 2376,
 2387,
 2403,
 2405,
 2409,
 2412,
 2415,
 2416,
 2431,
 2453,
 2454,
 2480,
 249

In [22]:
# Add the names of the zip files only to a list. 
local_zip_list = []

station_ids_selected = list(dfNRW.index)

for station_id in station_ids_selected:
    try:
        fname = df_zips["name"][station_id]
        print(fname)
        grabFile(ftp_dir + fname, local_ftp_ts_dir + fname)
        local_zip_list.append(fname)
    except:
        print("WARNING: TS file for key %d not found in FTP directory." % station_id)

tageswerte_RR_00029_19510101_20191231_hist.zip
tageswerte_RR_00046_19410101_20191231_hist.zip
tageswerte_RR_00055_19510101_20191231_hist.zip
tageswerte_RR_00057_19780101_20191231_hist.zip
tageswerte_RR_00063_19510101_20191231_hist.zip
tageswerte_RR_00103_19780101_20191231_hist.zip
tageswerte_RR_00118_19510101_20191231_hist.zip
tageswerte_RR_00128_19510101_20191231_hist.zip
tageswerte_RR_00142_19550101_20191231_hist.zip
tageswerte_RR_00145_19410101_20191231_hist.zip
tageswerte_RR_00146_19310101_20191231_hist.zip
tageswerte_RR_00151_19310101_20191231_hist.zip
tageswerte_RR_00160_19710101_20191231_hist.zip
tageswerte_RR_00163_19510101_20191231_hist.zip
tageswerte_RR_00202_19410101_20191231_hist.zip
tageswerte_RR_00203_19310101_20191231_hist.zip
tageswerte_RR_00205_19310101_20191231_hist.zip
tageswerte_RR_00217_19410101_20191231_hist.zip
tageswerte_RR_00223_19310101_20191231_hist.zip
tageswerte_RR_00227_19410101_20191231_hist.zip
tageswerte_RR_00232_19470101_20191231_hist.zip
tageswerte_RR

tageswerte_RR_02860_19410101_20191231_hist.zip
tageswerte_RR_02867_19510101_20191231_hist.zip
tageswerte_RR_02884_19410101_20191231_hist.zip
tageswerte_RR_02885_19310101_20191231_hist.zip
tageswerte_RR_02894_19310101_20191231_hist.zip
tageswerte_RR_02943_19310101_20191231_hist.zip
tageswerte_RR_03010_19360101_20191231_hist.zip
tageswerte_RR_03055_19310101_20191231_hist.zip
tageswerte_RR_03080_19710101_20191231_hist.zip
tageswerte_RR_03121_18990123_20191231_hist.zip
tageswerte_RR_03131_19310101_20191231_hist.zip
tageswerte_RR_03147_19510101_20191231_hist.zip
tageswerte_RR_03153_19410101_20191231_hist.zip
tageswerte_RR_03179_19121105_20191231_hist.zip
tageswerte_RR_03180_18990101_20191231_hist.zip
tageswerte_RR_03182_19410101_20191231_hist.zip
tageswerte_RR_03183_19310101_20191231_hist.zip
tageswerte_RR_03185_19310101_20191231_hist.zip
tageswerte_RR_03188_19010101_20191231_hist.zip
tageswerte_RR_03190_19510101_20191231_hist.zip
tageswerte_RR_03192_19310101_20191231_hist.zip
tageswerte_RR

tageswerte_RR_05015_19510101_20191231_hist.zip
tageswerte_RR_05017_19310101_20191231_hist.zip
tageswerte_RR_05023_19310101_20191231_hist.zip
tageswerte_RR_05036_19410101_20191231_hist.zip
tageswerte_RR_05046_19410101_20191231_hist.zip
tageswerte_RR_05054_19410101_20191231_hist.zip
tageswerte_RR_05055_19410101_20191231_hist.zip
tageswerte_RR_05065_19410101_20191231_hist.zip
tageswerte_RR_05073_19510101_20191231_hist.zip
tageswerte_RR_05111_19410101_20191231_hist.zip
tageswerte_RR_05123_19510101_20191231_hist.zip
tageswerte_RR_05141_19510101_20191231_hist.zip
tageswerte_RR_05149_18990101_20191231_hist.zip
tageswerte_RR_05160_19310101_20191231_hist.zip
tageswerte_RR_05161_19410101_20191231_hist.zip
tageswerte_RR_05162_19610101_20191231_hist.zip
tageswerte_RR_05167_19780101_20191231_hist.zip
tageswerte_RR_05169_19310101_20191231_hist.zip
tageswerte_RR_05176_19310101_20191231_hist.zip
tageswerte_RR_05187_19310101_20191231_hist.zip
tageswerte_RR_05216_19410101_20191231_hist.zip
tageswerte_RR

### Join (Merge) the Time Series Columns

https://medium.com/@chaimgluck1/working-with-pandas-fixing-messy-column-names-42a54a6659cd


In [23]:
def prec_ts_to_df(fname):
    
    dateparse = lambda dates: [pd.datetime.strptime(str(d), '%Y%m%d') for d in dates]

    df = pd.read_csv(fname, delimiter=";", encoding="utf8", index_col="MESS_DATUM", parse_dates = ["MESS_DATUM"], date_parser = dateparse, na_values = [-999.0, -999])

    #df = pd.read_csv(fname, delimiter=";", encoding="iso8859_2",\
    #             index_col="MESS_DATUM", parse_dates = ["MESS_DATUM"], date_parser = dateparse)
    
    # https://medium.com/@chaimgluck1/working-with-pandas-fixing-messy-column-names-42a54a6659cd

    # Column headers: remove leading blanks (strip), replace " " with "_", and convert to lower case.
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    df.index.name = df.index.name.strip().lower().replace(' ', '_').replace('(', '').replace(')', '')
    return(df)

In [24]:
#def temp_ts_to_df(fname):
    
#    dateparse = lambda dates: [pd.datetime.strptime(str(d), '%Y%m%d') for d in dates]

#    df = pd.read_csv(fname, delimiter=";", encoding="utf8", index_col="MESS_DATUM_BEGINN", parse_dates = ["MESS_DATUM_BEGINN"], date_parser = dateparse, na_values = [-999.0, -999])

    #df = pd.read_csv(fname, delimiter=";", encoding="iso8859_2",\
    #             index_col="MESS_DATUM", parse_dates = ["MESS_DATUM"], date_parser = dateparse)
    
    # https://medium.com/@chaimgluck1/working-with-pandas-fixing-messy-column-names-42a54a6659cd

    # Column headers: remove leading blanks (strip), replace " " with "_", and convert to lower case.
#    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
#    df.index.name = df.index.name.strip().lower().replace(' ', '_').replace('(', '').replace(')', '')
#    return(df)

In [25]:
from zipfile import ZipFile

In [26]:
def ts_merge():
    # Very compact code.
    df = pd.DataFrame()
    for elt in local_zip_list:
        ffname = local_ftp_ts_dir + elt
        print("Zip archive: " + ffname)
        with ZipFile(ffname) as myzip:
            # read the time series data from the file starting with "produkt"
            prodfilename = [elt for elt in myzip.namelist() if elt.split("_")[0]=="produkt"][0] 
            print("Extract product file: %s" % prodfilename)
            print()
            with myzip.open(prodfilename) as myfile:
                dftmp = prec_ts_to_df(myfile)
#                s = dftmp["r1"].rename(dftmp["stations_id"][0]).to_frame()
                s = dftmp["stations_id"].to_frame()
                # outer merge.
                df = pd.merge(df, s, left_index=True, right_index=True, how='outer')

    #df.index.names = ["year"]
    df.index.rename(name = "time", inplace = True)
    return(df)

In [27]:
#def prec_ts_merge():
    # Very compact code.
#    df = pd.DataFrame()
#    for elt in local_zip_list:
#        ffname = local_ftp_ts_dir + elt
#        print("Zip archive: " + ffname)
#        with ZipFile(ffname) as myzip:
            # read the time series data from the file starting with "produkt"
#            prodfilename = [elt for elt in myzip.namelist() if elt.split("_")[0]=="produkt"][0] 
#            print("Extract product file: %s" % prodfilename)
#            print()
#            with myzip.open(prodfilename) as myfile:
#                dftmp = temp_ts_to_df(myfile)
#                s = dftmp["ja_tt"].rename(dftmp["stations_id"][0]).to_frame()
                # outer merge.
#                df = pd.merge(df, s, left_index=True, right_index=True, how='outer')

    #df.index.names = ["year"]
#    df.index.rename(name = "time", inplace = True)
#    return(df)

In [28]:
df_merged_ts = ts_merge()

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00029_19510101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19510101_20191231_00029.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00046_19410101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20191231_00046.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00055_19510101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19510101_20191231_00055.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00057_19780101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19780101_20191231_00057.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00063_19510101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19510101_20191231_00063.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00103_19780101_20191231_hist.zip
Ex

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00801_19410101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20191231_00801.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00803_19410101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20191231_00803.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00807_19410101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20191231_00807.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00808_19410101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20191231_00808.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00811_19610101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19610101_20191231_00811.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00812_19610101_20191231_hist.zip
Ex

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_01528_19410101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20191231_01528.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_01534_19410101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20191231_01534.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_01542_19310101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20191231_01542.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_01550_19360101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19360101_20191231_01550.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_01575_19510101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19510101_20191231_01575.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_01577_19410101_20191231_hist.zip
Ex

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_02403_18960901_20191231_hist.zip
Extract product file: produkt_nieder_tag_18960901_20191231_02403.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_02405_19310101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20191231_02405.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_02409_19310101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20191231_02409.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_02412_19310101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20191231_02412.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_02415_19310101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20191231_02415.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_02416_19510101_20191231_hist.zip
Ex

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03180_18990101_20191231_hist.zip
Extract product file: produkt_nieder_tag_18990101_20191231_03180.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03182_19410101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20191231_03182.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03183_19310101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20191231_03183.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03185_19310101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20191231_03185.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03188_19010101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19010101_20191231_03188.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03190_19510101_20191231_hist.zip
Ex

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03621_19410101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20191231_03621.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03667_19750101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19750101_20191231_03667.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03668_18790101_20191231_hist.zip
Extract product file: produkt_nieder_tag_18790101_20191231_03668.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03670_19310101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20191231_03670.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03677_19710301_20191231_hist.zip
Extract product file: produkt_nieder_tag_19710301_20191231_03677.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03679_19410101_20191231_hist.zip
Ex

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04065_19410101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20191231_04065.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04086_19510101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19510101_20191231_04086.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04103_18990101_20191231_hist.zip
Extract product file: produkt_nieder_tag_18990101_20191231_04103.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04104_18790101_20191231_hist.zip
Extract product file: produkt_nieder_tag_18790101_20191231_04104.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04106_18990101_20191231_hist.zip
Extract product file: produkt_nieder_tag_18990101_20191231_04106.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04107_19410101_20191231_hist.zip
Ex

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04706_19510101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19510101_20191231_04706.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04754_19310101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20191231_04754.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04755_19510101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19510101_20191231_04755.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04759_19610101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19610101_20191231_04759.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04760_19510101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19510101_20191231_04760.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04808_19410101_20191231_hist.zip
Ex

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_05303_19410101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20191231_05303.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_05336_19410101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20191231_05336.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_05364_19310101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20191231_05364.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_05370_19340412_20191231_hist.zip
Extract product file: produkt_nieder_tag_19340412_20191231_05370.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_05374_19410101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20191231_05374.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_05376_19410101_20191231_hist.zip
Ex

In [29]:
df_merged_ts.head()

,stations_id_x,stations_id_y,stations_id_x,stations_id_y,stations_id_x,stations_id_y,stations_id_x,stations_id_y,stations_id_x,stations_id_y,...,stations_id_x,stations_id_y,stations_id_x,stations_id_y,stations_id_x,stations_id_y,stations_id_x,stations_id_y,stations_id_x,stations_id_y
time,,,,,,,,,,,,,,,,,,,,,
1781-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1781-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1781-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1781-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1781-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
#%matplotlib inline

In [31]:
#import matplotlib.pyplot as plt

In [32]:
#fig = plt.figure(dpi= 136, figsize=(8,4))
#ax1 = fig.add_subplot(221)
#ax2 = fig.add_subplot(222)
#ax3 = fig.add_subplot(223)
#ax4 = fig.add_subplot(224)
#df_merged_ts[1024].plot(ax = ax1)
#df_merged_ts[1232].plot(ax = ax1)
#df_merged_ts[1246].plot(ax = ax2)
#df_merged_ts[1277].plot(ax = ax3)
#df_merged_ts[1300].plot(ax = ax4)
#plt.show()



In [33]:
#import seaborn as sns
#import matplotlib.pyplot as plt
#%matplotlib inline

# plot
#sns.set_style('ticks')
#fig1, ax1 = plt.subplots(dpi = 400, figsize = (12,24))

#sns.heatmap(df_merged_ts, cmap='RdYlGn_r', annot=False, ax = ax1)
#sns.heatmap(df_merged_ts, cmap='coolwarm', annot=True, vmin = 8, vmax = 12, ax = ax1)

# _r reverses the normal order of the color map 'RdYlGn'

#sns.heatmap(df, cmap='coolwarm', annot=True, vmin = 8, vmax = 12, ax = ax)
#ax1.set_yticklabels(df_merged_ts.index.strftime('%Y'))
#plt.show()
#fig1.savefig('example1.png')

In [34]:
df_merged_ts.to_csv(local_ts_merged_dir + "ts_merged_prec.csv",sep=";")

In [35]:
df_merged_ts_transposed = df_merged_ts.transpose()

In [36]:
df_merged_ts_transposed.index.names = ['station_id']

In [37]:
df_merged_ts_transposed.head()

time,1781-01-01,1781-01-02,1781-01-03,1781-01-04,1781-01-05,1781-01-06,1781-01-07,1781-01-08,1781-01-09,1781-01-10,...,2019-12-22,2019-12-23,2019-12-24,2019-12-25,2019-12-26,2019-12-27,2019-12-28,2019-12-29,2019-12-30,2019-12-31
station_id,,,,,,,,,,,,,,,,,,,,,
stations_id_x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
stations_id_y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
stations_id_x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,55.0,55.0,55.0,55.0,55.0,55.0,55.0,55.0,55.0,55.0
stations_id_y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,57.0,57.0,57.0,57.0,57.0,57.0,57.0,57.0,57.0,57.0
stations_id_x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0


In [38]:
df_merged_ts_transposed.to_csv(local_ts_merged_dir + "ts_merged_transposed.csv",sep=";")

In [39]:
def ts_append():
    # Very compact code.
    df = pd.DataFrame()
    for elt in local_zip_list:
        ffname = local_ftp_ts_dir + elt
        print("Zip archive: " + ffname)
        with ZipFile(ffname) as myzip:
            # read the time series data from the file starting with "produkt"
            prodfilename = [elt for elt in myzip.namelist() if elt.split("_")[0]=="produkt"][0] 
            print("Extract product file: %s" % prodfilename)
            print()
            with myzip.open(prodfilename) as myfile:
                dftmp = prec_ts_to_df(myfile)
                dftmp=dftmp.loc[(dftmp.index>='2016-06-01 00:00:00')&(dftmp.index<='2020-06-01 00:00:00')]
                dftmp = dftmp.merge(df_stations,how="inner",left_on="stations_id",right_on="station_id",right_index=True)
#                print(dftmp.head(5))
                df = df.append(dftmp)

    #df.index.names = ["year"]
    #df.index.rename(name = "time", inplace = True)
    return(df)

In [ ]:
df_appended_ts = ts_append()

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00029_19510101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19510101_20191231_00029.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00046_19410101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20191231_00046.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00055_19510101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19510101_20191231_00055.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00057_19780101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19780101_20191231_00057.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00063_19510101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19510101_20191231_00063.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00103_19780101_20191231_hist.zip
Ex

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00801_19410101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20191231_00801.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00803_19410101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20191231_00803.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00807_19410101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20191231_00807.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00808_19410101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20191231_00808.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00811_19610101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19610101_20191231_00811.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_00812_19610101_20191231_hist.zip
Ex

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_01528_19410101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20191231_01528.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_01534_19410101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20191231_01534.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_01542_19310101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20191231_01542.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_01550_19360101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19360101_20191231_01550.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_01575_19510101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19510101_20191231_01575.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_01577_19410101_20191231_hist.zip
Ex

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_02403_18960901_20191231_hist.zip
Extract product file: produkt_nieder_tag_18960901_20191231_02403.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_02405_19310101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20191231_02405.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_02409_19310101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20191231_02409.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_02412_19310101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20191231_02412.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_02415_19310101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20191231_02415.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_02416_19510101_20191231_hist.zip
Ex

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03180_18990101_20191231_hist.zip
Extract product file: produkt_nieder_tag_18990101_20191231_03180.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03182_19410101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20191231_03182.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03183_19310101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20191231_03183.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03185_19310101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20191231_03185.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03188_19010101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19010101_20191231_03188.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03190_19510101_20191231_hist.zip
Ex

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03621_19410101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20191231_03621.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03667_19750101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19750101_20191231_03667.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03668_18790101_20191231_hist.zip
Extract product file: produkt_nieder_tag_18790101_20191231_03668.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03670_19310101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20191231_03670.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03677_19710301_20191231_hist.zip
Extract product file: produkt_nieder_tag_19710301_20191231_03677.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_03679_19410101_20191231_hist.zip
Ex

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04064_18990109_20191231_hist.zip
Extract product file: produkt_nieder_tag_18990109_20191231_04064.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04065_19410101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20191231_04065.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04086_19510101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19510101_20191231_04086.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04103_18990101_20191231_hist.zip
Extract product file: produkt_nieder_tag_18990101_20191231_04103.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04104_18790101_20191231_hist.zip
Extract product file: produkt_nieder_tag_18790101_20191231_04104.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04106_18990101_20191231_hist.zip
Ex

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04697_19510101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19510101_20191231_04697.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04706_19510101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19510101_20191231_04706.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04754_19310101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20191231_04754.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04755_19510101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19510101_20191231_04755.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04759_19610101_20191231_hist.zip
Extract product file: produkt_nieder_tag_19610101_20191231_04759.txt

Zip archive: data/original/DWD//daily/more_precip/historical/tageswerte_RR_04760_19510101_20191231_hist.zip
Ex

In [ ]:
df_appended_ts.head()

In [ ]:
df_appended_ts.to_csv(local_ts_appended_dir + "ts_appended_prec.csv",sep=";")